In [9]:

#necessary packages import
import pandas as pd
import numpy as np
import re
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB

In [10]:
#reading the excel file containing the dataset
df = pd.read_csv("../bible_data_set/bible_data_set.csv")
new = ['Matthew', 'Mark', 'Luke', 'John', 'Acts', 'Romans', '1 Corinthians',
            '2 Corinthians', 'Galatians', 'Ephesians', 'Philippians', 'Colossians',
            '1 Thessalonians', '2 Thessalonians', '1 Timothy', '2 Timothy', 'Titus', 'Philemon',
            'Hebrews', 'James', '1 Peter', '2 Peter', '1 John', '2 John', '3 John', 'Jude',
            'Revelation']
df['class'] = np.where(df['book'].isin(new), 1, 0)
df = df[['class','text']]
df.columns = ['label', 'data']

In [11]:

#data cleansing
def cleansing(x):
    x = x.lower()
    x = re.sub(r"http\S+", "", x)
    x = re.sub(r'[^\w\s]', '', x)
    x = " ".join(filter(lambda x: x[0] != '@' and x[0]!= '#', x.split()))
    x = " ".join(filter(lambda x: x.isalpha() , x.split()))
    x = re.sub(' +', ' ', x)
    return x.strip()
df['data'] = df['data'].apply(lambda x: cleansing(x))

In [12]:
#separate the feature set and label
df_x = df['data']
df_y = df['label']

In [13]:
#using TFIDF vectorizer for word to vector conversion
cv=TfidfVectorizer(min_df=1,stop_words='english')

In [14]:
#splitting the training and testing set
x_train,x_test,y_train,y_test=train_test_split(df_x,df_y,test_size=0.30,random_state=45)

In [15]:
#scaling the feature vectors for optimization
x_traincv = cv.fit_transform(x_train)
x_testcv = cv.transform(x_test)
y_train=y_train.astype('int')

In [16]:
#defining the model
svm_clf=svm.SVC(kernel='linear', C = 1.0)
#svm_clf = MultinomialNB()

In [22]:
#loading the data to the model with the label and training
def training():
    global x_traincv,y_train,svm_clf
    svm_clf.fit(x_traincv,y_train)

In [23]:
#accuracy prediction
def testing():
    global svm_clf
    clf1 = open('model/svm_tfidf.pickle','rb')
    svm_clf = pickle.load(clf1)
    pred=svm_clf.predict(x_testcv)
    actual=np.array(y_test)
    count_0=0
    count_1=0
    for i in range (len(pred)):
        if int(pred[i]) == int(actual.item(i)) and int(actual.item(i)) == 0:
          count_0=count_0+1
        if int(pred[i]) == int(actual.item(i)) and int(actual.item(i)) == 1:
          count_1=count_1+1
    print("The number of samples in the test set is",len(pred))
    print("The accuracy on the test set is---> ")      
    print("Accuracy-->",(count_1 + count_0)*100/len(pred))

In [24]:
#save the model using pickle
def model_saving():
    global svm_clf
    clf = open('model/svm_tfidf.pickle','wb')
    pickle.dump(svm_clf,clf)
    clf.close()

In [25]:
training()
model_saving()
testing()

The number of samples in the test set is 9331
The accuracy on the test set is---> 
Accuracy--> 92.03729503804523
